In [1]:
import numpy as np
import pandas as pd
import json
import re

In [2]:
with open('data/daterangegovinfo02.json', 'r') as fl:
    govinfo_dct = json.loads(fl.read())

In [8]:
text_arr = [el['transcript'] for el in govinfo_dct]

clean_text_arr = []
CLEANR = re.compile('<.*?>')
CLEANR2 = re.compile('\[.*?\]')
CLEANR3 = re.compile("\s+")

for text in text_arr:
    temp = re.sub(CLEANR, '', text)
    temp = re.sub(CLEANR2, '', temp)
    temp = temp.replace('\n', ' ')
    temp = re.sub(CLEANR3, ' ', temp).strip()
    clean_text_arr.append(temp)

text_arr = clean_text_arr.copy()
del clean_text_arr
print(text_arr[0])

Congressional Record, Volume 168 Issue 2 (Tuesday, January 4, 2022) From the Congressional Record Online through the Government Publishing Office Remembering Johnny Isakson Mr. THUNE. Mr. President, before I begin, I want to express my deep sorrow at the death of former Senator Johnny Isakson. I served with Johnny for a long time, not just here in the Senate, where we came in together as freshman Members in 2005, but also in the House of Representatives, and I was honored to call him my friend. Johnny was a tremendously effective legislator and, in particular, a champion for veterans and a model of decency and graciousness. When I came down to the floor to discuss his retirement 2 years ago, I read a quote from a politics professor in Georgia that I thought really captured Johnny. I think so still. Here is what that professor had to say: As a political science professor and an administrator, I'm often asked by students if good people can serve in government and keep their integrity. Jo

In [14]:
rolldf = pd.read_csv('data/H118_rollcalls.csv')
bills = list(set(rolldf['bill_number']))
bills[2]

'HR593'

In [15]:
rolldf.head()

,congress,chamber,rollnumber,date,session,clerk_rollnumber,yea_count,nay_count,nominate_mid_1,nominate_mid_2,nominate_spread_1,nominate_spread_2,nominate_log_likelihood,bill_number,vote_result,vote_desc,vote_question,dtl_desc
0,118,House,1,2023-01-03,1,2,212,222,0.023,-0.045,0.716,-0.133,-0.352,NaN,Jeffries,NaN,Election of the Speaker,NaN
1,118,House,2,2023-01-03,1,3,212,222,0.023,-0.045,0.716,-0.133,-0.352,NaN,Jeffries,NaN,Election of the Speaker,NaN
2,118,House,3,2023-01-03,1,4,212,222,0.023,-0.045,0.716,-0.133,-0.352,NaN,Jeffries,NaN,Election of the Speaker,NaN
3,118,House,4,2023-01-04,1,5,212,221,0.023,-0.045,0.716,-0.133,-0.352,NaN,Jeffries,NaN,Election of the Speaker,NaN
4,118,House,5,2023-01-04,1,6,212,221,0.023,-0.045,0.716,-0.133,-0.352,NaN,Jeffries,NaN,Election of the Speaker,NaN


In [12]:
data_arr = []
for text in text_arr:
    for bill in bills:
        if str(bill) in text:
            if 
            data_arr.append(
                [
                    text, max(rolldf[rolldf['bill_number'] == bill]['yea_count'])
                 ]
            )

204